In [1]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore')
import scalp
from scalp.output import draw
import lmz
import numpy as np

ImportError: cannot import name 'Iterable' from 'collections' (/home/ikea/.myconda/miniconda3/envs/scalp/lib/python3.11/collections/__init__.py)

In [ ]:
import pandas as pd
pd.__version__

In [ ]:
datasets = scalp.data.loaddata_scib(scalp.test_config.scib_datapath,  maxdatasets=10,  maxcells = 1000 )
dataset = datasets[2]
dataset = dataset[:5]

# scanorama

# UMAP ONLY 

In [ ]:
import scanpy as sc
def umaponly(dataset,dim=2):
    dataset = scalp.pca.pca(dataset)
    [ds.obsm.pop('umap', None)for ds in dataset]
    scalp.umapwrap.adatas_umap(dataset, label ='umap', from_obsm ='pca40', dim =dim)
    return scalp.transform.stack(dataset)
    
stack = umaponly(dataset)
sc.pl.umap(stack, color=['batch', 'label'])

# SCALP

In [ ]:
reducer = pacmap.PaCMAP(n_components=2,n_neighbors=10, MN_ratio=0.5, FP_ratio=5.0)
# fit the data (The index of transformed data corresponds to the index of the original data)
stack = scalp.transform.stack(dataset)
X = stack.X.todense()
X_transformed = reducer.fit_transform(X, init="pca")
stack.obsm['umap']= X_transformed
sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:
import scanpy as sc
from scalp import graph as sgraph
from scipy.sparse import csr_matrix

def Scalp(dataset, dim = 2):
    num_ds = len(dataset)
    print(num_ds)
    parm = {'neighbors_total': 30, 'intra_neighbors_mutual': False,
            'neighbors_intra_fraction': .2,  # is ignored when only 1 guy is batch
            'add_tree': False, "epsilon":1e-4,
                  'copy_lsa_neighbors': False, 
            'inter_outlier_threshold': 0.7, 
            'inter_outlier_probabilistic_removal': False}
  
    dataset, grap = scalp.mkgraph(dataset,**parm)

    
    # scalp.umapwrap.graph_pacmap2(dataset,csr_matrix(grap),label = 'umap', n_components = dim)
    #scalp.umapwrap.graph_pacmap(dataset,grap, label = 'umap', n_components = dim)
    #scalp.umapwrap.graph_xxumap(dataset,grap, label = 'umap', n_components = dim)
    # scalp.umapwrap.graph_umap(dataset,grap,label = 'umap', n_components = dim)
    
    #plt.matshow(grap.todense())
    return scalp.transform.stack(dataset), grap

#stack, grap= Scalp([dataset[0]])
stack, grap= Scalp(dataset)
#sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:
# plt.matshow((grap@grap).todense())

In [ ]:
data = (grap > 0).sum(axis =1)
asd = np.sort(data, axis =0)
plt.plot(asd)



In [ ]:
import pacmap
from ubergauss import tools as ut
if False:
    reducer = pacmap.PaCMAP(n_components=2, n_neighbors=10, MN_ratio=0.5, FP_ratio=2.0) 
    # fit the data (The index of transformed data corresponds to the index of the original data)
    X = dataset[0].X.todense()
    y=  list(dataset[0].obs['label'])
    y, _ = ut.labelsToIntList(y)
    X_transformed = reducer.fit_transform(X, init="pca")
    # visualize the embedding
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.scatter(X_transformed[:, 0], X_transformed[:, 1], cmap="Spectral", c=y, s=0.6)


In [ ]:
dataset

In [ ]:
from scalp.data.transform import stack_single_attribute, attach_stack

g2 = grap > 0
g3 = g2 @ g2
#for i in range(1): g3 = g3@g2

#plt.matshow((g3).todense())

def choosefrom(csr, invert=False, ratio = 1):
    csr = csr_matrix(csr)
    n_neighbors = 10
    choose = int(n_neighbors * ratio)
    n = csr.shape[0]
    pairs = np.empty((n*choose, 2), dtype=np.int32)
    range = np.arange(csr.shape[1])
    for i,row in enumerate(csr): 
        id = row.indices
        if invert:
            id = np.setdiff1d(range, id)
        np.random.shuffle(id)
        candidates = id[:choose+4]
        candidates = candidates[candidates!=i]
        for j, idd in enumerate(candidates[:choose]):
            pairs[i*choose+j] = i,idd
    return pairs
    
def choosefrom_dekirudake(csr, invert=False, ratio = 1):
    csr = csr_matrix(csr)
    range = np.arange(csr.shape[1])
    pairs = []
    
    for i,row in enumerate(csr): 
        # prepare our candidates: 
        id = row.indices
        if invert:
            id = np.setdiff1d(range, id)
        np.random.shuffle(id)
        candidates = id[:ratio+1]
        candidates = candidates[candidates!=i]
        # just add until we hit the limit
        for j, idd in enumerate(candidates):
            pairs.append((i,idd))
    
    return np.array(pairs,dtype=np.int32)

from sklearn.neighbors import NearestNeighbors
def choose_medium_pairs(dataset,ratio = 5):
    offset = 0
    pairs = []
    for adata in dataset:
        le = adata.X.shape[0]
        start = int(le*.10)
        end = int(le*.25)
        _, ids = NearestNeighbors(n_neighbors= end).fit(adata.X).kneighbors(adata.X)
        ids = ids[:,start:]
        # now we sample and assign with the offset :D DOTO!!!!!!!!!
        for i, row in enumerate(ids):
            np.random.shuffle(row)
            row=row[:ratio+1]
            pairs += [(i+offset,r+offset) for r in row]
            
        
        offset += adata.X.shape[0]

    return np.array(pairs,dtype=np.int32)
    
# neighs = choosefrom(g2)
# mn = choosefrom (g3-g2, ratio = .5)
# far = choosefrom( g3, invert='True', ratio=2) 
neighs = choosefrom_dekirudake(g2, ratio = 10)
#mn = choosefrom_dekirudake (g3-g2, ratio = 5)
mn = choose_medium_pairs(dataset, ratio = 5)
far = choosefrom_dekirudake( g3, invert='True', ratio=100) 

def plausible(blob,neighpairs):
    allowed = blob.shape[0]
    for pairs in neighpairs:
        if np.max(pairs) >= allowed:
            print('not plausible')
            return False
    return True
    
if plausible(grap, [neighs,mn,far]):
    reducer = pacmap.PaCMAP(n_components=2, #n_neighbors=10, MN_ratio=0.5, FP_ratio=2.0,
                     pair_neighbors=neighs,
                     pair_MN=mn, 
                     pair_FP=far ) 
    
    # fit the data (The index of transformed data corresponds to the index of the original data)
    X = stack.X.todense()
    X_transformed = reducer.fit_transform(X, init="pca")
    stack.obsm['umap']= X_transformed
    sc.pl.umap(stack, color=['batch', 'label'])

In [ ]:
plt.matshow((g3-g2).todense())

In [ ]:
g2.todense()

In [ ]:
g3.todense()

In [ ]:
import pacmap
import numpy as np
import matplotlib.pyplot as plt
from annoy import AnnoyIndex

# loading preprocessed coil_20 dataset

# create nearest neighbor pairs
# here we use AnnoyIndex as an example, but the process can be done by any
# external NN library that provides neighbors into a matrix of the shape
# (n, n_neighbors_extra), where n_neighbors_extra is greater or equal to
# n_neighbors in the following example.
from scipy.sparse import csr_matrix
from ubergauss import graphumap 

X , di = graphumap.make_knn(csr_matrix(grap))
nbrs=X
n,n_neighbors = X.shape

scaled_dist = np.ones((n, n_neighbors)) # No scaling is needed
# Type casting is needed for numba acceleration
X = X.astype(np.float32)
scaled_dist = scaled_dist.astype(np.float32)
# make sure n_neighbors is the same number you want when fitting the data
pair_neighbors = pacmap.sample_neighbors_pair(X.astype(np.float32), scaled_dist.astype(np.float32), nbrs.astype(np.int32), np.int32(n_neighbors))
# initializing the pacmap instance
# feed the pair_neighbors into the instance
embedding = pacmap.PaCMAP(n_components=2, n_neighbors=n_neighbors, MN_ratio=0.5, FP_ratio=2.0, pair_neighbors=pair_neighbors) 

# fit the data (The index of transformed data corresponds to the index of the original data)
X_transformed = embedding.fit_transform(X, init="pca")

# visualize the embedding
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.scatter(X_transformed[:, 0], X_transformed[:, 1], cmap="Spectral", c=y, s=0.6)

In [ ]:

def make_dis_plot(saveas, numgraphs, distribution, similarity):
    """
    """
    rc = {'color': 'r'}
    bc = {'color': 'b'}
    ws = 2.2
    o = np.mean(distribution, axis=1)
    o = np.median(distribution, axis=1)
    s = np.mean(similarity, axis=1)
    s = np.median(similarity, axis=1)
    print(s,o)
    plt.figure(figsize=(18, 8))
    marksize = 5

    # fontsize...
    fsa = 13
    fsb = 15

    # OKOK NEW STUFF TESTING
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    for label in (ax1.get_xticklabels() + ax1.get_yticklabels()):
        #label.set_fontname('Arial')
        label.set_fontsize(fsa)
    for label in (ax2.get_xticklabels() + ax2.get_yticklabels()):
        #label.set_fontname('Arial')
        label.set_fontsize(fsa)
    numgraphs = np.array(numgraphs)

    # plt.grid()
    ax1.set_axisbelow(True)
    ax1.yaxis.grid(color='gray', linestyle='dashed')

    ax1.boxplot(distribution, positions=numgraphs, widths=ws, capprops=bc, medianprops=bc, boxprops=bc, whiskerprops=bc,
                flierprops=None)
    ax2.boxplot(similarity, positions=numgraphs, widths=ws, capprops=rc, medianprops=rc, boxprops=rc, whiskerprops=rc,
                flierprops=None)
    ax1.plot(numgraphs, o, color='b', marker='o', markeredgewidth=1, markersize=marksize, markeredgecolor='b',
             markerfacecolor='w', label='KL divergence', linewidth=2)
    ax2.plot(numgraphs, s, color='r', marker='o', markeredgewidth=1, markersize=marksize, markeredgecolor='r',
             markerfacecolor='w', label='similarity', linewidth=2)

    # plt.xlim(percentages[0]-.05,percentages[-1]+.05)
    plt.xlim(min(numgraphs) - 2, max(numgraphs) + 2)
    ax1.set_ylim(0, .1)
    ax2.set_ylim(.9, 1)
    ax1.set_xlim(20, 105)
    ax2.set_xlim(20, 105)
    plt.xticks(numgraphs, numgraphs)

    # plt.title(run_id + '\n', fontsize=18)
    ax1.legend(loc='lower left', fontsize=fsa)
    ax2.legend(loc='upper right', fontsize=fsa)
    # plt.ylabel('ROC AUC',fontsize=18)
    ax1.set_ylabel('divergence', fontsize=fsb)
    ax2.set_ylabel('similarity', fontsize=fsb)
    ax2.set_xlabel('training sequences', fontsize=fsb)
    ax1.set_xlabel('training sequences', fontsize=fsb)
    # plt.savefig('%d_displot.png' % saveas)
    plt.show()


numgraph =  [20, 50, 100, 200, 400]
distr=[[0.25545721593226361,0.000522490030742645,0.028502527247154782,0.050284436793470791,0.12003533257090437,0.35243988950184624,0.093199909425701583],[0.0043773373094962067,0.024590936307660054,0.01084206338719987,0.0017994298202424208,0.0048688732759201419,0.0050635384688497015,0.00016076222467320198],[0.00039921348927913092,0.00089444792182533962,0.0022715091525810165,0.00097458591203185198,0.016969225115156028,0.0032465116886065881,0.0020030658974116234],[5.1610560071823665e-05,0.005678508129071092,0.00019214440765663414,4.5874999054601635e-05,0.00010455603767192245,0.0023978066568669629,0.0056785190178299891],[0.00063615124933100262,9.7782400071463188e-06,3.0495939638172567e-05,0.00017389088174254801,1.213987689807584e-05,0.0010211143036889352,0.00049004822293609383]]
simi=[[0.89368195647572157,0.95448263387898369,0.95567463656141338,0.93408416962747653,0.93081546003093063,0.95488685043168364,0.95518695427485956],[0.98195457433876521,0.97705392644211408,0.97683284903317469,0.98408056995350346,0.98246265189501181,0.98163296634212505,0.97947557919767625],[0.98971139219482929,0.98541645250279986,0.98696360643624348,0.98890080279435277,0.98582165004771305,0.98715798773211916,0.98948882878201327],[0.9933786312922358,0.99278323741893237,0.99283697184835396,0.9916789742797848,0.99265274352623534,0.99251407139394809,0.99312405150635652],[0.99347677829781555,0.99444865104153124,0.99469145661591951,0.99372163858407636,0.99485476032036058,0.99480043701636556,0.9938539331627807]]
make_dis_plot(0,numgraph,distr,simi)



In [ ]:
import numpy as np

# Create a NumPy array with numbers between 0 and 1000
array = np.random.randint(0, 1001, size=10000)

# Count occurrences of each number
counts = np.bincount(array)

# Identify the numbers to keep (those that occur 50 times or fewer)
numbers_to_keep = np.where(counts <= 50)[0]

# Create a mask for the elements to keep
mask = np.isin(array, numbers_to_keep)

# Filter the array using the mask
filtered_array = array[mask]

In [ ]:
counts.shape

In [ ]:
from sklearn.neighbors import NearestNeighbors
x = dataset[0].X.todense().A
neighbors = 10
model = NearestNeighbors(n_neighbors=neighbors*2).fit(x)
distances, indices = model.kneighbors(x)
counts = np.bincount(indices.flatten())
counts_mat = counts[indices]
cnt_srt = np.argsort(counts_mat, axis = 1)
indices_new = np.take_along_axis(indices, cnt_srt, axis =1)[:,:neighbors]
neighborsgraph = np.zeros((x.shape[0],x.shape[0]))
np.put_along_axis(neighborsgraph,indices_new,1,axis=1)

In [ ]:
print(counts_mat[:10])
print(indices[:10])
print(indices_new[:10])


In [ ]:
plt.matshow(neighborsgraph)